In [21]:
import numpy as np 
import pandas as pd
import re
import nltk
from sklearn.metrics import accuracy_score, confusion_matrix , classification_report
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer ,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import string
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
df = pd . read_csv("Downloads\\Train_Dataset.csv")
display (df.head() , df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44262 entries, 0 to 44261
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      44262 non-null  object
 1   is_sarcastic  44262 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 691.7+ KB


,headline,is_sarcastic
0,supreme court votes 7-2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily's list founder: women are the 'problem s...,0
3,send your kids back to school with confidence,0
4,watch: experts talk pesticides and health,0


None

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\EGYPT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\EGYPT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
corpus = np.array (df["headline"]) 
corpus

array(['supreme court votes 7-2 to legalize all worldly vices',
       'hungover man horrified to learn he made dozens of plans last night',
       "emily's list founder: women are the 'problem solvers' in congress",
       ...,
       'protesters ejected from donald trump rally after holding up pocket constitutions',
       'italian recipes that are oldies but goodies',
       'area loser blissfully unaffected by whims of stock market'],
      dtype=object)

In [5]:
stop_words = nltk.corpus.stopwords.words("english")
def normalize(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    doc = ' '.join(filtered_tokens)
    return doc


normal = np.vectorize(normalize)
ready_corpus = normal(corpus)
len(ready_corpus)

44262

In [6]:
df

,headline,is_sarcastic
0,supreme court votes 7-2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily's list founder: women are the 'problem s...,0
3,send your kids back to school with confidence,0
4,watch: experts talk pesticides and health,0
...,...,...
44257,greece seeks to reassure europe as tensions rise,0
44258,vatican says transgender man cannot become a g...,0
44259,protesters ejected from donald trump rally aft...,0
44260,italian recipes that are oldies but goodies,0


In [7]:
xtrain , xtest , ytrain ,ytest = train_test_split(df['headline'] 
                                                  , df["is_sarcastic"])
xtrain.shape , ytest.shape

((33196,), (11066,))

In [8]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1)),
                   
               ])
logreg.fit (xtrain,ytrain)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
  

In [23]:
svm = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('SVC', SVC()),
                   
               ])
svm.fit (xtrain,ytrain)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('SVC',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,

# Model Evaluation
Just accuracy is never enough in datasets with a rare class problem.

--Precision: The positive predictive power of a model. Out of all the predictions made by a model for a class, how many are actually correct

--Recall: The coverage or hit-rate of a model. Out of all the test data samples belonging to a class, how many was the model able to predict (hit or cover) correctly.


In [17]:
y_pred = logreg.predict(xtest)
print('accuracy %s' % accuracy_score(y_pred, ytest))
print(classification_report(ytest, y_pred))

accuracy 0.8815290077715525
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      5998
           1       0.87      0.87      0.87      5068

    accuracy                           0.88     11066
   macro avg       0.88      0.88      0.88     11066
weighted avg       0.88      0.88      0.88     11066



In [16]:
score = cross_val_score(logreg , X = xtrain , y=ytrain , cv = 20 , n_jobs= 1)
score

array([0.87349398, 0.8813253 , 0.86144578, 0.8753012 , 0.88192771,
       0.8813253 , 0.86084337, 0.87710843, 0.8813253 , 0.88554217,
       0.86144578, 0.88313253, 0.85783133, 0.86807229, 0.87590361,
       0.8813253 , 0.88125377, 0.88547318, 0.8848704 , 0.87763713])

In [28]:
y_pred = svm.predict(xtest)
print('accuracy %s' % accuracy_score(y_pred, ytest))
print(classification_report(ytest, y_pred))

accuracy 0.9322248328212543
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5998
           1       0.93      0.93      0.93      5068

    accuracy                           0.93     11066
   macro avg       0.93      0.93      0.93     11066
weighted avg       0.93      0.93      0.93     11066



In [27]:
scoresvm = cross_val_score(svm , X = xtrain , y=ytrain , cv = 20 , n_jobs= 1)
scoresvm

array([0.9246988 , 0.93253012, 0.91566265, 0.92831325, 0.92590361,
       0.93493976, 0.91385542, 0.92048193, 0.93192771, 0.93253012,
       0.91626506, 0.92831325, 0.91445783, 0.92228916, 0.92108434,
       0.93433735, 0.92163954, 0.93248945, 0.93429777, 0.91681736])

###### TEST ON TESTSET

In [31]:
new_test_data = pd.read_csv("Downloads\Test_Dataset.csv" )

target = svm.predict(new_test_data.squeeze())
target

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [32]:
res = pd.DataFrame(target) 
res.index = new_test_data.index 
res.columns = ["prediction"]
res.to_csv("prediction_results.csv", index = False)  